In [1]:
# j'importe mes librairies :

import pandas as pd
import numpy as np

In [2]:
# J'importe mes fichiers de données au format .csv :

aide_alim = pd.read_csv(r'C:\Users\chouc\Documents\Farid\Data Analyst _ OpenClassrooms\4 - Projet 4\Mission _ Réalisez une étude de santé publique avec R ou Python\DAN-P4-FAO\aide_alimentaire.csv')
dispo_alim = pd.read_csv(r'C:\Users\chouc\Documents\Farid\Data Analyst _ OpenClassrooms\4 - Projet 4\Mission _ Réalisez une étude de santé publique avec R ou Python\DAN-P4-FAO\dispo_alimentaire.csv')
pop = pd.read_csv(r'C:\Users\chouc\Documents\Farid\Data Analyst _ OpenClassrooms\4 - Projet 4\Mission _ Réalisez une étude de santé publique avec R ou Python\DAN-P4-FAO\population.csv')
sous_nut = pd.read_csv(r'C:\Users\chouc\Documents\Farid\Data Analyst _ OpenClassrooms\4 - Projet 4\Mission _ Réalisez une étude de santé publique avec R ou Python\DAN-P4-FAO\sous_nutrition.csv')

# 1)  Demandes de Marc

**1.1) Pour l'année 2017 : la proportion de personnes en état de sous-nutrition**

In [4]:
# Je remplace les fourchettes d'années par des années :

sous_nut.replace(['2012-2014', '2013-2015', '2014-2016', '2015-2017', '2016-2018', '2017-2019'],['2013', '2014', '2015', '2016', '2017', '2018'],inplace=True)

# Je supprime les NaN pour pouvoir effetuer des calculs :

sous_nut.fillna((0),inplace=True)

# Je remplace les valeurs '<0.1' en '0.1' afin de pouvoir  changer le type de donnée :

sous_nut['Valeur'].replace({'<0.1':0.1}, inplace = True)

# Je change le type de données de la colonne 'valeur' du df "sous_nut" : 

sous_nut['Valeur']= sous_nut['Valeur'].astype('float64')

# Je modifie le type de données de la colonne "Année" du df "sous_nut" afin de ne sélectionner que l'année 2017 :

sous_nut['Année'] = sous_nut['Année'].astype(int)

# Je sélectionne l'année 2017 : 

sous_nut_2017 = sous_nut[(sous_nut.Année == 2017)]   

# Je selectionne la population de l'année 2017, et j'en fais la somme pour connaitre le nombre d'habitant dans le monde en 2017 
# dans le df population (X 1000 car les données sont en milliers):

pop_2017 = sum(pop.loc[pop['Année']==2017, 'Valeur'])*1000

# J'affiche le résultat de ma première opération :

print(f'En 2017, il y avait {round(pop_2017)} personnes sur Terre.')

# J'affiche le nombre de personne en sous nutrition en 2017 : 

sous_nut_2017 = sum(sous_nut.loc[sous_nut['Année']==2017, 'Valeur'])*1000000

print(f'En 2017, il y avait {round(sous_nut_2017)} personnes en sous nutrition sur Terre.')

# Je calcule la proportion de personne en état de sous nutrition en 2017 :

prop_ssnut_2017 = (sous_nut_2017) / (pop_2017) *100

# J'arrondis le résultat : 

prop_ssnut_2017 = round(prop_ssnut_2017,2)

# J'affiche le résultat : 

print(f'En 2017, il y avait {round(pop_2017)} personnes sur Terre, et cette même année, {prop_ssnut_2017}% de la population, soit {round(sous_nut_2017)} personnes, étaient en état de sous nutrition.')

En 2017, il y avait 7548134111 personnes sur Terre.
En 2017, il y avait 537700000 personnes en sous nutrition sur Terre.
En 2017, il y avait 7548134111 personnes sur Terre, et cette même année, 7.12% de la population, soit 537700000 personnes, étaient en état de sous nutrition.


**1.2) Pour l'année 2017 : le nombre théorique de personnes qui pourraient être nourries**

*Note : nombre de calories quotidiennes nécessaires pour nourrir une personne adulte = 2400 kcal*

In [5]:
# J'isole l'année 2017 du df "population" :

pop_2017 = pop.loc[pop['Année']==2017]

# Je fait une jointure des df de la "pop2017" et "disponibilité alimentaire" :

dispo_alim = pd.merge(dispo_alim, pop_2017, how='left', on='Zone')

# Je fais la somme des populations de chaque pays pour obtenir la population mondiale de 2017 : 

pop_2017_sum = pop_2017['Valeur'].sum()*1000

# Je créé une colonne permettant de calculer la disponibilité alimentaires journalière pour la population de chaque pays : 

dispo_alim['kcal/jour/population'] = dispo_alim['Valeur'] * dispo_alim['Disponibilité alimentaire (Kcal/personne/jour)']

# Je fais la somme des colonnes créés que je divise par 2400 (nombre de calories quotidiennes nécessaires pour nourrir
# une personne adulte que j'ai estimé).
# Le résultat obtenu est en milliers, c'est pourquoi je multiplie par 1000

resultat = round(dispo_alim['kcal/jour/population'].sum()/2400*1000)

# J'arrondis et j'affiche le résultat :

print(f'La disponibilité alimentaire mondiale de 2017 pouvait nourrir {resultat} personnes, soit {round(resultat / pop_2017_sum,2)} fois la population mondiale')

La disponibilité alimentaire mondiale de 2017 pouvait nourrir 8716243595 personnes, soit 1.15 fois la population mondiale


**1.3) Pour l'année 2017 : le nombre théorique de personnes qui pourraient être nourries pour la disponibilité alimentaire des produits végétaux**

*Note : nombre de calories quotidiennes nécessaires pour nourrir une personne adulte = 2400 kcal*

In [6]:
# J'isole l'année 2017 du df "population" :

pop_2017 = pop.loc[pop['Année']==2017]

# je fais une jointure des df de la "pop_2017" et "disponibilité alimentaire" :

dispo_alim_2 = pd.merge(dispo_alim, pop_2017, how='left', on='Zone')

# Je définis le df 'dispo_alim_vege'

dispo_alim_vege = dispo_alim_2.loc[dispo_alim_2['Origine'] == 'vegetale']

# je joins les df "pop2017" et "dispo_alim_vege" :

dispo_alim_vege = pd.merge(dispo_alim_vege, pop_2017, how='left', on='Zone')

# Je fais la somme des populations de chaque pays pour obtenir la population mondiale de 2017 : 

pop_2017_sum = pop_2017['Valeur'].sum()*1000

# Je créé une colonne permettant de calculer la disponibilité alimentaires journalière pour la population de chaque pays : 

dispo_alim_vege['kcal_vege/jour/population'] = dispo_alim_vege['Valeur'] * dispo_alim_vege['Disponibilité alimentaire (Kcal/personne/jour)']

# je fais la somme des colonnes créés que je divise par 2400, Le résultat obtenu est en milliers, c'est pourquoi 
# jemultiplie par 1000 : 

resultat_vege = round(dispo_alim_vege['kcal_vege/jour/population'].sum()/2400*1000)

# J'arrondir et j'affiche le résultat

print(f'La disponibilité alimentaire vegetale mondiale de 2017 pouvait nourrir {resultat_vege} personnes, soit {round(resultat_vege / pop_2017_sum,2)} fois la population mondiale')

La disponibilité alimentaire vegetale mondiale de 2017 pouvait nourrir 7191985088 personnes, soit 0.95 fois la population mondiale


**1.4) Pour l'année 2017 : l’utilisation de la disponibilité intérieure, en particulier :<br>
    - la part qui est attribuée à l’alimentation animale,<br>
    - celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine.**

In [8]:
# je calcule la disponibilité intérieur totale :

dis_int_tot = round(dispo_alim['Disponibilité intérieure'].sum())

# Je calcule la part attribuée aux animaux : 

a_p_an = round(dispo_alim['Aliments pour animaux'].sum())

# Je calcule la proportion : "part attribuée à l’alimentation animale / disponibilité interieur" :

prop_part_anim = round((a_p_an / dis_int_tot)*100)

# J'affiche le résultat obtenu :

print(f'La part de disponibilité intérieur attribuée à l’alimentation animale est de {prop_part_anim}%')

# Je calcule la part de disponibilité intérieure perdue :

pertes_da = round(dispo_alim['Pertes'].sum())

# Je calcul la proportion : " pertes / disponibilité interieur " :

prop_pertes = round((pertes_da / dis_int_tot)*100)

# J'affiche le résultat obtenu :

print(f'La proportion de pertes de la disponibilité alimentaire est de {prop_pertes}%')


# Je créé un df avec les variables nécessaires à afficher :

pcuah = dispo_alim[['Disponibilité intérieure', 
                     'Exportations - Quantité',
                    'Importations - Quantité',
                    'Nourriture',
                    'Pertes',
                    'Production',
                    'Semences',
                    'Traitement',
                    'Variation de stock',
                    'Aliments pour animaux',
                    'Autres Utilisations']]

# Je fais la somme des variables de mon nouveau df :

pcuah2 = pcuah.sum()

# Je créé une nouvelle colonneque j''alimente par le calcul suggérer par Julien : 

#part_humain = pcuah2['Disponibilité intérieure'] - pcuah2['Pertes'] - pcuah2['Aliments pour animaux'] - pcuah2['Semences'] - pcuah2['Traitement'] - pcuah2['Autres Utilisations']

part_humain = pcuah2['Disponibilité intérieure'] - pcuah2['Pertes'] - pcuah2['Aliments pour animaux'] - pcuah2['Semences'] - pcuah2['Traitement'] - pcuah2['Autres Utilisations']

# Je calcule en % la part concrètement utilisée pour l'alimentation humaine :

resultat_4 = part_humain/dispo_alim['Disponibilité intérieure'].sum()*100

# J'affiche le résultat obtenu :

print(f'La proportion de disponibilité intérieure concrètement utilisée pour l\'alimentation humaine est de {round(resultat_4)} % de la disponibilitée intérieur')

La part de disponibilité intérieur attribuée à l’alimentation animale est de 13%
La proportion de pertes de la disponibilité alimentaire est de 5%
La proportion de disponibilité intérieure concrètement utilisée pour l'alimentation humaine est de 49 % de la disponibilitée intérieur


# 2) Demandes de Mélanie 

**2.1) Pour l'année 2017 : les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte**

In [9]:
# Je créé un df pour afficher la population en sous-nutrition dans le monde pour l'année 2017 :

sous_nuth =  sous_nut.loc[sous_nut['Année']==2017]
sous_nuth

# Je créé un df pour afficher la population mondiale l'année 2017 :

poph =  pop.loc[pop['Année']==2017]
poph

# Je joins les deux df créés :

merge1 = pd.merge(poph, sous_nuth, how='outer', left_on=['Zone','Année'], right_on=['Zone','Année'], suffixes=('_pop_tot', '_ss_nut_tot'))
merge1

# Je ramène la valeur du dela variable "valeur_ss_nut_tot" sous_nuth en millions pour le calcul :

merge1['Valeur_ss_nut_tot'] = merge1['Valeur_ss_nut_tot']*1000000
merge1

# Je ramène la valeur du dela variable "Valeur_pop_tot" sous_nuth en milliers pour le calcul :

merge1['Valeur_pop_tot'] = merge1['Valeur_pop_tot']*1000
merge1

# créé une nouvelle colonne qui affiche la proportion de personnes en sous-nutrition par pays : 

merge1['prop_sn'] =  merge1['Valeur_ss_nut_tot'] / merge1['Valeur_pop_tot']
merge1

# J'exclus les pays de moins d'un millions d'habitants pour avoir un résultat plus parlant : 

merge1 = merge1.loc[merge1['Valeur_pop_tot']> 1000000]
merge1

# Je fait un tri par ordre décroissant sur la variable "prop_sn", et j'affiche les 10 premiers résultats : 

merge1 = merge1.sort_values(by = 'prop_sn', ascending = False) [0:10]

# Je supprime les colonnes inutiles : 

merge1 = merge1.drop(['Année', 'Valeur_pop_tot', 'Valeur_ss_nut_tot'], axis=1)

# J'affiche le résultat : 

merge1

,Zone,prop_sn
87,Haïti,0.482592
181,République populaire démocratique de Corée,0.471887
128,Madagascar,0.410629
122,Libéria,0.382797
119,Lesotho,0.382494
216,Tchad,0.379576
186,Rwanda,0.350556
145,Mozambique,0.328109
219,Timor-Leste,0.321735
0,Afghanistan,0.289287


**2.2) Les pays qui ont le plus bénéficié d’aide depuis 2013**

In [10]:
# Je supprime les colonnes inutiles : 

aide_alim = aide_alim.drop('Année', axis=1)

# je regroupe les pays bénéficiaires et je fait la somme des valeurs : 

aide_alim_sum = aide_alim.groupby('Pays bénéficiaire').sum()

# Je trie le résultat précedent par ordre décroissant, et je prends les 10 premières valeurs :

aide_alim_sum = aide_alim_sum.sort_values(by = 'Valeur', ascending = False) [0:10]

# j'affiche le résultat :

aide_alim_sum

,Valeur
Pays bénéficiaire,
République arabe syrienne,1858943
Éthiopie,1381294
Yémen,1206484
Soudan du Sud,695248
Soudan,669784
Kenya,552836
Bangladesh,348188
Somalie,292678
République démocratique du Congo,288502


**2.3) Les pays ayant le plus/le moins de disponibilité/habitant**

In [14]:
# j'affiche de le df dispo alim pour la question, en en selectionnant que la colonne Disponibilité intérieure :

dispo_alim3 = dispo_alim.groupby('Zone').sum().loc[:,['Disponibilité alimentaire (Kcal/personne/jour)']]

# Je fais la moyenne des habitants par pays sur la plage d'année disponible :

popq6 = pop.groupby('Zone').sum()

# Je fais une jointure entre les tables "popq6" & "dispo_alim3" : 

dfq6 = popq6.merge(dispo_alim3, on='Zone')

# Je trie la variable "Disponibilité alimentaire (Kcal/personne/jour)" par ordre décroissant, 
# et je sélectionne les 10 premières valeurs :

dispo_hab =  dfq6.sort_values(by = 'Disponibilité alimentaire (Kcal/personne/jour)', ascending = True) [0:10]

# Je supprime les colonnes inutiles :

dispo_hab = dispo_hab.drop(['Année', 'Valeur'], axis=1)

# J'affiche le résulat obtenu :

dispo_hab

,Disponibilité alimentaire (Kcal/personne/jour)
Zone,
République centrafricaine,1879.0
Zambie,1924.0
Madagascar,2056.0
Afghanistan,2087.0
Haïti,2089.0
République populaire démocratique de Corée,2093.0
Tchad,2109.0
Zimbabwe,2113.0
Ouganda,2126.0
